In [46]:
#https://www.analyticsvidhya.com/blog/2021/06/a-detailed-explanation-of-gan-with-implementation-using-tensorflow-and-keras/
from numpy import zeros, ones, expand_dims, asarray
from numpy.random import randn, randint
from keras.models import Sequential, load_model,Model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization, Activation
from keras import initializers
from keras.initializers import RandomNormal
from matplotlib import pyplot
import numpy as np
from math import sqrt
import pickle

In [47]:
#chargement des données
with open('dataset_tau_array', 'rb') as f:
    X_train = pickle.load(f)
X_train = np.array(X_train)

In [48]:
with open('nom_colonne','rb') as f:
    nom_col = pickle.load(f)

In [49]:
#1500 prot de 2598 aa avec 21 aa diff
X_train.shape

(1500, 2598, 21)

In [50]:
#On vient générer une entrée de nombre aléatoire pour que le generator tente de les transformer en image
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)  
    z_input = x_input.reshape(n_samples, latent_dim)
    return z_input

In [51]:
# On ajoute aux données réelles la classe 1 ce qui permet au discriminator de savoir que les données sont réelles lors du fitting
def generate_real_samples(X_train, n_samples):
    ix = randint(0, len(X_train), n_samples) 
    X = X_train[ix]  
    y = ones((n_samples, 1)) 
    return X, y

In [52]:
#on génère de fausses images avec la classe 0 ce qui permet au discriminator de savoir que les données sont fausses lors du fitting
def generate_fake_samples(generator, latent_dim, n_samples):
    z_input = generate_latent_points(latent_dim, n_samples)
    images = generator.predict(z_input)  
    y = zeros((n_samples, 1))
    return images, y

In [53]:
#on définie les couches du discriminator
def define_discriminator(in_shape=(2598, 21, 1)):
    model = Sequential()
    model.add(Input(shape=in_shape))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [54]:
discriminator = define_discriminator()

In [55]:
#on définie les couches du generator
def define_generator(latent_dim): 
    model_g = Sequential()
    model_g.add(Input(shape=(latent_dim,)))
    model_g.add(Dense(256))
    model_g.add(LeakyReLU(alpha=0.2))
    model_g.add(Dense(512))
    model_g.add(LeakyReLU(alpha=0.2))
    model_g.add(Dense(1024))
    model_g.add(LeakyReLU(alpha=0.2))
    model_g.add(Dense(2598 * 21 * 1,activation='sigmoid'))
    model_g.add(Reshape((2598, 21, 1)))
    model_g.compile(loss='categorical_crossentropy',optimizer = 'Adam')
    return model_g

In [56]:
generator = define_generator(100)

In [57]:
#On créé le GAN contenant discriminator et generator
def define_gan(g_model, d_model):
    #quand on entraine le GAN le discriminator ne s'entraine pas seulement le generator
    d_model.trainable = False
    gan_output = d_model(g_model.output)
    model = Model(g_model.input, gan_output)
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [58]:
gan_model = define_gan(generator, discriminator)

In [59]:
#fonction d'entrainement
def train(g_model, d_model, gan_model, X_train, latent_dim, n_epochs=10, n_batch=64):
    bat_per_epo = int(len(X_train) / n_batch)
    n_steps = bat_per_epo * n_epochs
    for i in range(n_steps):
        #on entraine le discriminator
        X_real, y_real = generate_real_samples(X_train, n_batch)
        d_loss_r, d_acc_r = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
        d_loss_f, d_acc_f = d_model.train_on_batch(X_fake, y_fake)
        # on entraine le generator (le discriminator ne peut pas être entrainé via gan_model)
        z_input = generate_latent_points(latent_dim, n_batch) 
        y_gan = ones((n_batch, 1)) 
        g_loss, g_acc = gan_model.train_on_batch(z_input, y_gan)
        #suivis des loss et acc pour s'assurer que le modèle s'entraine
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_loss_r,d_acc_r, d_loss_f,d_acc_f, g_loss,g_acc))
    #on sauvegarde le generative model pour une future utilisation
    g_model.save('./exemple_GAN_model/model_GAN')

In [61]:
#entrainement du GAN
latent_dim = 100
train(generator, discriminator, gan_model, X_train, latent_dim, n_epochs=10, n_batch=64)

>1, dr[0.000,1.000], df[0.000,1.000], g[11.857,0.000]
>2, dr[0.000,1.000], df[0.010,1.000], g[12.052,0.000]
>3, dr[0.000,1.000], df[0.000,1.000], g[12.953,0.000]
>4, dr[0.000,1.000], df[0.001,1.000], g[13.042,0.000]
>5, dr[0.001,1.000], df[0.000,1.000], g[13.689,0.000]
>6, dr[0.002,1.000], df[0.000,1.000], g[13.732,0.000]
>7, dr[0.002,1.000], df[0.000,1.000], g[14.752,0.000]
>8, dr[0.000,1.000], df[0.000,1.000], g[14.050,0.000]
>9, dr[0.001,1.000], df[0.000,1.000], g[14.517,0.000]
>10, dr[0.000,1.000], df[0.000,1.000], g[14.343,0.000]
>11, dr[0.002,1.000], df[0.000,1.000], g[13.777,0.000]
>12, dr[0.000,1.000], df[0.000,1.000], g[14.226,0.000]
>13, dr[0.001,1.000], df[0.000,1.000], g[13.155,0.000]
>14, dr[0.001,1.000], df[0.000,1.000], g[12.753,0.000]
>15, dr[0.000,1.000], df[0.000,1.000], g[13.299,0.000]
>16, dr[0.000,1.000], df[0.000,1.000], g[12.724,0.000]
>17, dr[0.000,1.000], df[0.001,1.000], g[12.604,0.000]
>18, dr[0.000,1.000], df[0.000,1.000], g[13.003,0.000]
>19, dr[0.000,1.000

>150, dr[0.000,1.000], df[0.000,1.000], g[15.604,0.000]
>151, dr[0.000,1.000], df[0.000,1.000], g[15.343,0.000]
>152, dr[0.001,1.000], df[0.000,1.000], g[15.544,0.000]
>153, dr[0.000,1.000], df[0.000,1.000], g[16.701,0.000]
>154, dr[0.000,1.000], df[0.000,1.000], g[15.719,0.000]
>155, dr[0.000,1.000], df[0.000,1.000], g[16.362,0.000]
>156, dr[0.000,1.000], df[0.000,1.000], g[16.065,0.000]
>157, dr[0.000,1.000], df[0.000,1.000], g[15.522,0.000]
>158, dr[0.000,1.000], df[0.000,1.000], g[15.885,0.000]
>159, dr[0.000,1.000], df[0.000,1.000], g[15.175,0.000]
>160, dr[0.000,1.000], df[0.000,1.000], g[15.280,0.000]
>161, dr[0.000,1.000], df[0.000,1.000], g[16.012,0.000]
>162, dr[0.000,1.000], df[0.000,1.000], g[15.098,0.000]
>163, dr[0.000,1.000], df[0.000,1.000], g[15.078,0.000]
>164, dr[0.000,1.000], df[0.000,1.000], g[15.850,0.000]
>165, dr[0.000,1.000], df[0.000,1.000], g[15.265,0.000]
>166, dr[0.000,1.000], df[0.000,1.000], g[16.170,0.000]
>167, dr[0.000,1.000], df[0.000,1.000], g[16.028

In [66]:
#test de création de prot avec le generator
model = load_model('./exemple_GAN_model/model_GAN')
latent_dim = 100
n_examples = 1
latent_points = generate_latent_points(latent_dim, n_examples)
X  = model.predict(latent_points)

In [67]:
#transforme nos données en séquences protéiques
def to_sequence(prot_array,nom_col) :
    prot_aa = []
    for i in range(prot_array.shape[0]) :
        maxi = 0
        index = 0
        for j in range(prot_array[i,:,:].shape[0]) :
            if prot_array[i,j,0] > maxi :
                maxi = prot_array[i,j,:]
                index = j
        prot_aa.append(nom_col[index])
        prot_aa = [i for i in prot_aa if i != '-']
    return ''.join(prot_aa)

In [68]:
liste_seq = []
for i in X :
    liste_seq.append(to_sequence(i,nom_col))

In [69]:
liste_seq

['MAEPRQEFDVMEDHRQTLQDQEGDKDHGLKESPLQTPADDGSEEPGSETSDAKSTPTEDTVEGGAAAHEIETEEAGIGDTPNLEDQAAGHVTQAMIQPSETGKECLGSQASKFRKKGADGKPGTKIATPRGAAPPGQKGSANATRIPAKTTPSPKTPPGPYGESPKSGDRSGYSSPGSPGTPGSRSRTPSLPTPPTREPKKVAVVRTPPKSPSSAKSRLQTAPVPMPDLKNVKSKIGSTENLKHQPGGGKQIINLDSVQSCGSKDNIKHVPGGGSVQIVYKPVDLSKVTSKCGSLGNIHHKPGGGQVEVKSEKLDFKDRVQSKIGSLDNITHVPGGGNKKIETHKLTFRENAKAKTDHGAEIVYKSPVVSGDTSPRHLSNVSSTGSIDMVDSPQLATLADEVSASLAKQGL']